In [2]:
import graphlab as gl

Read web session date

In [3]:
# read some web session
data = gl.SFrame('session.sf')

[INFO] 1453284685 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Library/Python/2.7/site-packages/certifi/cacert.pem
1453284685 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to mkacenak@csob.cz and will expire on February 15, 2016. For commercial licensing options, visit https://turi.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-469 - Server binary: /Library/Python/2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1453284685.log
[INFO] GraphLab Server Version: 1.7.1


In [4]:
data.num_rows()

355909

### Viewing data

I can emit several commands to see that we are working with a fairly tame dataset.  After all, we only have five columns.  

In [5]:
data.head(3)

Columns:
	guid	str
	date	str
	time	str
	country	str
	browser	str
	url	str
	query	str
	domain	str
	top_domain	str
	ip	str

Rows: 3

Data:
+-------------------------------+------------+----------+---------+---------+
|              guid             |    date    |   time   | country | browser |
+-------------------------------+------------+----------+---------+---------+
| ibic=89C2874366054211A7DC3... | 2014-06-08 | 00:00:28 |    DE   |    CH   |
| ibic=3EB3304A0267409BA6148... | 2014-06-08 | 00:00:28 |    DE   |    CH   |
| ibic=09CF6B52D105462DB9CD8... | 2014-06-08 | 00:00:29 |    DE   |    FF   |
+-------------------------------+------------+----------+---------+---------+
+-------------------------------+-------------------------------+
|              url              |             query             |
+-------------------------------+-------------------------------+
| https://bankingportal.spar... | campaign=000817&bic=14338b... |
| https://www.stargames.net/... | campaign=000817&bic=14325f... |
| http://www.youtube.com/use... | campaign=000817&bic=143f7b... |
+-------------------------------+-------------------------------+
+-------------------------------+---------------------------+------+
|             domain            |         top_domain        |  ip  |
+-------------------------------+---------------------------+------+
| bankingportal.sparkasse-ha... | sparkasse-haslach-zell.de | 3608 |
|       www.stargames.net       |       stargames.net       | 5308 |
|        www.youtube.com        |        youtube.com        | 7053 |
+-------------------------------+---------------------------+------+
[3 rows x 10 columns]

In [6]:
data.show()

Canvas is accessible via web browser at the URL: http://localhost:49843/index.html
Opening Canvas in default web browser.


### ETL and statistics

Alright, I want a little more out of this SFrame. I want to add a few columns. Let's split the date and time columns into thier components.

In [7]:


def extract_time_fields(data):
  time_splits = data.split()[0].split(':')
  return {
    'hour': int(time_splits[0]),
    'minute': int(time_splits[1]),
    'second': int(time_splits[2])
  }

time_splits = data['time'].apply(lambda x: extract_time_fields(x))
data.add_columns(time_splits.unpack())

data.head(3)

Columns:
	guid	str
	date	str
	time	str
	country	str
	browser	str
	url	str
	query	str
	domain	str
	top_domain	str
	ip	str
	X.hour	int
	X.minute	int
	X.second	int

Rows: 3

Data:
+-------------------------------+------------+----------+---------+---------+
|              guid             |    date    |   time   | country | browser |
+-------------------------------+------------+----------+---------+---------+
| ibic=89C2874366054211A7DC3... | 2014-06-08 | 00:00:28 |    DE   |    CH   |
| ibic=3EB3304A0267409BA6148... | 2014-06-08 | 00:00:28 |    DE   |    CH   |
| ibic=09CF6B52D105462DB9CD8... | 2014-06-08 | 00:00:29 |    DE   |    FF   |
+-------------------------------+------------+----------+---------+---------+
+-------------------------------+-------------------------------+
|              url              |             query             |
+-------------------------------+-------------------------------+
| https://bankingportal.spar... | campaign=000817&bic=14338b... |
| https://www.stargames.net/... | campaign=000817&bic=14325f... |
| http://www.youtube.com/use... | campaign=000817&bic=143f7b... |
+-------------------------------+-------------------------------+
+-------------------------------+---------------------------+------+--------+----------+
|             domain            |         top_domain        |  ip  | X.hour | X.minute |
+-------------------------------+---------------------------+------+--------+----------+
| bankingportal.sparkasse-ha... | sparkasse-haslach-zell.de | 3608 |   0    |    0     |
|       www.stargames.net       |       stargames.net       | 5308 |   0    |    0     |
|        www.youtube.com        |        youtube.com        | 7053 |   0    |    0     |
+-------------------------------+---------------------------+------+--------+----------+
+----------+
| X.second |
+----------+
|    28    |
|    28    |
|    29    |
+----------+
[3 rows x 13 columns]

# Usage by country

In [8]:
import graphlab.aggregate as agg

user_count = data.groupby(key_columns=['country','guid'],operations=agg.COUNT()) \
                 .groupby(key_columns='country', operations=agg.COUNT()) \
                 .topk('Count')
print user_count

+---------+-------+
| country | Count |
+---------+-------+
|    DE   |  5994 |
|    IR   |   12  |
|    RO   |   10  |
|    PL   |   10  |
|    ES   |   9   |
|    CH   |   8   |
|    US   |   8   |
|    IT   |   7   |
|    GR   |   7   |
|    FR   |   7   |
+---------+-------+
[10 rows x 2 columns]



#Graph Analytics

In [9]:
g = gl.SGraph()
g = g.add_edges(data, src_field='guid', dst_field='domain')
g = g.add_edges(data, src_field='domain', dst_field='guid')
cc = gl.connected_components.create(g)

print "Connected components summary:\n", cc.summary()

PROGRESS: +-----------------------------+
PROGRESS: | Number of components merged |
PROGRESS: +-----------------------------+
PROGRESS: | 23080                       |
PROGRESS: | 0                           |
PROGRESS: +-----------------------------+
Connected components summary:
Class                                   : ConnectedComponentsModel

Graph
-----
num_edges                               : 711818
num_vertices                            : 22985

Results
-------
graph                                   : SGraph. See m['graph']
component size                          : SFrame. See m['component_size']
number of connected components          : 50
vertex component id                     : SFrame. See m['componentid']

Metrics
-------
training time (secs)                    : 8.5129

Queryable Fields
----------------
graph                                   : A new SGraph with the color id as a vertex property
component_id                            : An SFrame with each vertex's com

In [10]:
cc['component_size'].topk('Count')

component_id,Count
206,22879
4417,4
1656,3
1259,3
9576,3
9047,3
22,3
4925,3
490,2
1572,2


In [11]:
largest_component_id = cc['component_size'].topk('Count', k=1)['component_id'][0]

In [12]:
data = data.join(cc['component_id'], on={'domain':'__id'})
data.head(3)

Columns:
	guid	str
	date	str
	time	str
	country	str
	browser	str
	url	str
	query	str
	domain	str
	top_domain	str
	ip	str
	X.hour	int
	X.minute	int
	X.second	int
	component_id	int

Rows: 3

Data:
+-------------------------------+------------+----------+---------+---------+
|              guid             |    date    |   time   | country | browser |
+-------------------------------+------------+----------+---------+---------+
| ibic=89C2874366054211A7DC3... | 2014-06-08 | 00:00:28 |    DE   |    CH   |
| ibic=3EB3304A0267409BA6148... | 2014-06-08 | 00:00:28 |    DE   |    CH   |
| ibic=09CF6B52D105462DB9CD8... | 2014-06-08 | 00:00:29 |    DE   |    FF   |
+-------------------------------+------------+----------+---------+---------+
+-------------------------------+-------------------------------+
|              url              |             query             |
+-------------------------------+-------------------------------+
| https://bankingportal.spar... | campaign=000817&bic=14338b... |
| https://www.stargames.net/... | campaign=000817&bic=14325f... |
| http://www.youtube.com/use... | campaign=000817&bic=143f7b... |
+-------------------------------+-------------------------------+
+-------------------------------+---------------------------+------+--------+----------+
|             domain            |         top_domain        |  ip  | X.hour | X.minute |
+-------------------------------+---------------------------+------+--------+----------+
| bankingportal.sparkasse-ha... | sparkasse-haslach-zell.de | 3608 |   0    |    0     |
|       www.stargames.net       |       stargames.net       | 5308 |   0    |    0     |
|        www.youtube.com        |        youtube.com        | 7053 |   0    |    0     |
+-------------------------------+---------------------------+------+--------+----------+
+----------+--------------+
| X.second | component_id |
+----------+--------------+
|    28    |     206      |
|    28    |     206      |
|    29    |     206      |
+----------+--------------+
[3 rows x 14 columns]

In [13]:
g = gl.SGraph()
g = g.add_edges(data[data['component_id'] == largest_component_id], src_field='guid', dst_field='domain')
g = g.add_edges(data[data['component_id'] == largest_component_id], src_field='domain', dst_field='guid')
pr = gl.pagerank.create(g)
pr_out = pr.get('pagerank')

print "Best webites\n", pr_out.topk('pagerank' , 10)
print "Worst websites\n", pr_out.topk('pagerank' , 10, reverse=True)

PROGRESS: Counting out degree
PROGRESS: Done counting out degree
PROGRESS: +-----------+-----------------------+
PROGRESS: | Iteration | L1 change in pagerank |
PROGRESS: +-----------+-----------------------+
PROGRESS: | 1         | 28935.7               |
PROGRESS: | 2         | 17099.3               |
PROGRESS: | 3         | 13200                 |
PROGRESS: | 4         | 11187.2               |
PROGRESS: | 5         | 9504.93               |
PROGRESS: | 6         | 8078.45               |
PROGRESS: | 7         | 6866.37               |
PROGRESS: | 8         | 5836.32               |
PROGRESS: | 9         | 4960.81               |
PROGRESS: | 10        | 4216.68               |
PROGRESS: | 11        | 3584.17               |
PROGRESS: | 12        | 3046.53               |
PROGRESS: | 13        | 2589.55               |
PROGRESS: | 14        | 2201.12               |
PROGRESS: | 15        | 1870.95               |
PROGRESS: | 16        | 1590.31               |
PROGRESS: | 17        |

In [14]:
#displaying the  graph
subgraph = gl.SGraph()

subgraph = subgraph.add_edges(data[data['top_domain'] == '1001spiele.de'],
                              src_field='ip', dst_field='domain')

#subgraph = subgraph.add_edges(data[data['top_domain'] == '1001spiele.de'],
#                              src_field='domain', dst_field='top_domain')

subgraph.show(vlabel='id', highlight=['1001spiele.de'])

Canvas is updated and available in a tab in the default browser.


##Frequent pattern mining

In [15]:
# Make a train-test split.
train, test = data.random_split(0.9)

# Build a frequent pattern miner model.
model = gl.frequent_pattern_mining.create(train, 
                                          'top_domain', 
                                          features=['guid'], 
                                          min_length=2, 
                                          max_patterns=100)

PROGRESS: Indexing complete. Found 12920 unique items.
PROGRESS: Preprocessing complete. Found 6109 unique transactions.
PROGRESS: Building frequent pattern tree.
PROGRESS: +-----------+----------------+------------+----------------------+------------------+
PROGRESS: | Iteration | Num. Patterns  | Support    | Current Min Support  | Elapsed Time     |
PROGRESS: +-----------+----------------+------------+----------------------+------------------+
PROGRESS: | 0         | 0              | 3016       | 13                   | 163us            |
PROGRESS: | 1         | 0              | 2780       | 13                   | 226us            |
PROGRESS: | 2         | 1              | 2122       | 13                   | 296us            |
PROGRESS: | 3         | 4              | 2121       | 13                   | 388us            |
PROGRESS: | 4         | 11             | 1084       | 13                   | 600us            |
PROGRESS: | 5         | 26             | 849        | 13             

In [16]:
# Obtain the most frequent patterns.
patterns = model.get_frequent_patterns()
patterns.print_rows(max_column_width=150, num_rows=20)

+------------------------------------------------------+---------+
|                       pattern                        | support |
+------------------------------------------------------+---------+
|              [google.de, facebook.com]               |   1475  |
|               [google.de, youtube.com]               |   1335  |
|              [google.de, akamaihd.net]               |   1333  |
|             [facebook.com, youtube.com]              |   1129  |
|             [facebook.com, akamaihd.net]             |   976   |
|             [youtube.com, akamaihd.net]              |   901   |
|        [google.de, facebook.com, youtube.com]        |   771   |
|       [google.de, facebook.com, akamaihd.net]        |   673   |
|        [google.de, youtube.com, akamaihd.net]        |   635   |
|                [facebook.com, v9.com]                |   538   |
|      [facebook.com, youtube.com, akamaihd.net]       |   504   |
|                 [google.de, v9.com]                  |   490

In [17]:
# Make predictions based on frequent patterns.
predictions = model.predict(test)
print predictions.head()

PROGRESS: Preprocessing complete. Found 5021 unique transactions.
+-------------------------------+-----------------------------+----------------+
|              guid             |            prefix           |   prediction   |
+-------------------------------+-----------------------------+----------------+
| ibic=A13287C222714B61B6D7A... |              []             |  [google.de]   |
| ibic=A28C2F85CB8C4D55BCB96... |        [facebook.com]       |  [google.de]   |
| ibic=C0D7B6C032774CAC82309... |        [facebook.com]       |  [google.de]   |
| ibic=DF54D3D2914A48EB89F1B... |              []             |  [google.de]   |
| ibic=3BBE5C72DD314FF18A134... | [youtube.com, facebook.com] |  [google.de]   |
| ibic=41C02CAAB0E94FCABA21E... |        [youtube.com]        |  [google.de]   |
| ibic=BA800BC8974C4927B9354... |              []             |  [google.de]   |
| ibic=31EF5475A0D8410C8B297... |         [google.de]         | [facebook.com] |
| ibic=A52727A3DDC446C4B255B... |   [omdigo

# Predicting user behavior

In [18]:
model = gl.recommender.create(data, user_id='ip', item_id='domain')

PROGRESS: Recsys training: model = item_similarity
PROGRESS: Warning: Ignoring columns guid, date, time, country, browser, url, query, top_domain, X.hour, X.minute, X.second, component_id;
PROGRESS:     To use one of these as a target column, set target = <column_name>
PROGRESS:     and use a method that allows the use of a target.
PROGRESS: Preparing data set.
PROGRESS:     Data has 355909 observations with 7792 users and 16851 items.
PROGRESS:     Data prepared in: 0.615358s
PROGRESS: Computing item similarity statistics:
PROGRESS: Computing most similar items for 16851 items:
PROGRESS: +-----------------+-----------------+
PROGRESS: | Number of items | Elapsed Time    |
PROGRESS: +-----------------+-----------------+
PROGRESS: | 1000            | 4.86399         |
PROGRESS: | 2000            | 5.01065         |
PROGRESS: | 3000            | 5.15626         |
PROGRESS: | 4000            | 5.30441         |
PROGRESS: | 5000            | 5.42787         |
PROGRESS: | 6000            | 

In [19]:
recommend_result = model.recommend(users=None, k=5)
recommend_result.head()

PROGRESS: recommendations finished on 1000/7792 queries. users per second: 987.096
PROGRESS: recommendations finished on 2000/7792 queries. users per second: 972.906
PROGRESS: recommendations finished on 3000/7792 queries. users per second: 1015.85
PROGRESS: recommendations finished on 4000/7792 queries. users per second: 1019.59
PROGRESS: recommendations finished on 5000/7792 queries. users per second: 1033.27
PROGRESS: recommendations finished on 6000/7792 queries. users per second: 1075.03
PROGRESS: recommendations finished on 7000/7792 queries. users per second: 1096.91


ip,domain,score,rank
3608,web.de,0.108513861287,1
3608,logout.webde.uimserv.net,0.102819147129,2
3608,www.rscommunity.de,0.0833333333333,3
3608,www.runescape.com,0.0833333333333,4
3608,www.omamaturetube.com,0.0833333333333,5
5308,www.google.de,0.0701340783163,1
5308,www.youtube.com,0.0656101259882,2
5308,asrv-a.akamaihd.net,0.064869976893,3
5308,apps.facebook.com,0.0599767186714,4
5308,s.m2pub.com,0.0477826843677,5


In [20]:
model.get_similar_items(['web.de'])

PROGRESS: Getting similar items completed in 0.002222


domain,similar,score,rank
web.de,navigator.web.de,0.704318936877,1
web.de,logout.webde.uimserv.net,0.434782608696,2
web.de,3c.web.de,0.293436293436,3
web.de,produkte.web.de,0.109756097561,4
web.de,www.web.de,0.100694444444,5
web.de,de.reimageplus.com,0.0944881889764,6
web.de,suche.web.de,0.0698529411765,7
web.de,www.ebay.de,0.0504672897196,8
web.de,signin.ebay.de,0.0473372781065,9
web.de,asrv-a.akamaihd.net,0.0451586655818,10


# Detecting sex websites

In [21]:
data['xxx'] = data['top_domain'].apply( \
        lambda x: 1 if (x.find('xxx') != -1 or x.find('porn') != -1 or x.find('sex') != -1) else 0)

print data['xxx'].sum(), ' out of ', len(data), ' ', data['xxx'].sum() / float(len(data))

7899  out of  355909   0.0221938754007


# Finding similar users

In [22]:


data2 = data.groupby(['guid','top_domain'], gl.aggregate.COUNT()) \
            .groupby(['guid'], gl.aggregate.CONCAT('top_domain', 'Count'))
    
data2 = data2.add_row_number()
    
data2.print_rows(3, max_column_width=120)

+----+-----------------------------------------+
| id |                   guid                  |
+----+-----------------------------------------+
| 0  | ibic=A13287C222714B61B6D7AD45122A0135IE |
| 1  | ibic=A28C2F85CB8C4D55BCB9613EFB0271EDIE |
| 2  | ibic=C0D7B6C032774CAC823095D04381B015IE |
+----+-----------------------------------------+
+-------------------------------------------------------------------------------------------------------------------------+
|                                                 Dict of top_domain_Count                                                |
+-------------------------------------------------------------------------------------------------------------------------+
| {'cpmterra.com': 1, 'snapdo.com': 5, 'erotikchat4free.de': 2, 'akamaihd.net': 2, 'fuq.com': 4, 'imagefap.com': 1, 'p... |
| {'linesandsplines.com': 1, 't-online.de': 9, 'lpmxp1001.com': 1, 'srv123.com': 1, 'akamaihd.net': 3, '148casinos.com... |
| {'ligatus.com': 2, 'vuupc.com': 1, 

In [23]:
nn_model = gl.nearest_neighbors.create(data2, features=['Dict of top_domain_Count'])

PROGRESS: Starting brute force nearest neighbors model training.


In [24]:
# Find users who visit many porn sites
user_xxx_counts = data.groupby('guid', gl.aggregate.SUM('xxx')).sort('Sum of xxx', ascending=False)
user_xxx_counts.head(3)

guid,Sum of xxx
ibic=75802351FF90468CBE805B481CDF2611IE ...,211
ibic=DC6B2E3D103D402E8AC7CF17F5C0728DIE ...,191
ibic=86C5254CA86F4D4CA9333BADCB53E21CIE ...,182


In [25]:
out = nn_model.query(data2.filter_by(user_xxx_counts.head(3)['guid'], 'guid'))
print out

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 3       | 0.0163026   | 2.371ms      |
PROGRESS: | Done         |         | 100         | 10.895ms     |
PROGRESS: +--------------+---------+-------------+--------------+
+-------------+-----------------+----------------+------+
| query_label | reference_label |    distance    | rank |
+-------------+-----------------+----------------+------+
|      0      |       388       |      0.0       |  1   |
|      0      |       4590      | 0.860465116279 |  2   |
|      0      |       3316      | 0.870175438596 |  3   |
|      0      |       2246      | 0.881278538813 |  4   |
|      0      |       1695      | 0.912621359223 |  5   |
|      1      |       3443      |      0.0       |  1   |
|      1      |       895       | 0.94964028

In [26]:
out.join(data2, on={'query_label':'id'}).join(data2,on={'reference_label':'id'}).print_rows(3,max_column_width=100)

+-------------+-----------------+---------------+------+-----------------------------------------+
| query_label | reference_label |    distance   | rank |                   guid                  |
+-------------+-----------------+---------------+------+-----------------------------------------+
|      0      |       388       |      0.0      |  1   | ibic=A13287C222714B61B6D7AD45122A0135IE |
|      1      |       388       | 0.95971563981 |  4   | ibic=A28C2F85CB8C4D55BCB9613EFB0271EDIE |
|      1      |       895       | 0.94964028777 |  2   | ibic=A28C2F85CB8C4D55BCB9613EFB0271EDIE |
+-------------+-----------------+---------------+------+-----------------------------------------+
+-----------------------------------------------------------------------------------------------------+
|                                       Dict of top_domain_Count                                      |
+--------------------------------------------------------------------------------------------------